In [1]:
__author__ = "Ermakov Petr"
__license__ = "GPL"
__version__ = "1.0"
__email__ = "ermakov.pd@gmail.com"

In [2]:
!pip install nltk

Импортируем необходимые библиотеки

In [3]:
import pandas as pd
from IPython.display import display, Math

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import numpy as np
import scipy.sparse as sp

from nltk.text import TextCollection

Инициализируем массив ```texts```, который содержит в себе 3 элемента - 3 'синтетических' текста

In [4]:
texts = [u'мама мама мама мыла рама',
         u'рама это рама все просто',
         u'очень просто']

# Считаем частотность

Без использования дополнительных библиотек посчитаем частотность каждого слова по всем текстам

In [6]:
word_to_number = {}
for text in texts:
    for word in text.split():
        word_to_number[word] = word_to_number.get(word, 0) + 1

Для красивой визуализации использум pandas, не обязательно, но наглядно

In [7]:
pd.DataFrame(list(word_to_number.items()))

,0,1
0,мама,3
1,мыла,1
2,рама,3
3,это,1
4,все,1
5,просто,2
6,очень,1


Без использования дополнительных библиотек посчитаем частотность каждого слова по каждому тексту

In [8]:
word_to_number_for_each_texts = {}
for text in texts:
    word_to_number_for_each_texts[text] = {}
    for word in text.split():
        if word_to_number_for_each_texts[text].get(word):
            word_to_number_for_each_texts[text][word] += 1
        else:
            word_to_number_for_each_texts[text][word] = 1

In [9]:
word_to_number_for_each_texts

{'мама мама мама мыла рама': {'мама': 3, 'мыла': 1, 'рама': 1},
 'очень просто': {'очень': 1, 'просто': 1},
 'рама это рама все просто': {'все': 1, 'просто': 1, 'рама': 2, 'это': 1}}

Для красивой визуализации использум pandas, не обязательно, но наглядно (в этот раз для красивой визуализации нужно заморочиться, но мы не будем заморачиваться)

In [10]:
pd.DataFrame(list(word_to_number_for_each_texts.items()))

,0,1
0,мама мама мама мыла рама,"{'мама': 3, 'мыла': 1, 'рама': 1}"
1,рама это рама все просто,"{'рама': 2, 'это': 1, 'все': 1, 'просто': 1}"
2,очень просто,"{'очень': 1, 'просто': 1}"


In [11]:
pd.DataFrame(list(word_to_number_for_each_texts.values())).fillna(0)

,все,мама,мыла,очень,просто,рама,это
0,0.0,3.0,1.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0,2.0,1.0
2,0.0,0.0,0.0,1.0,1.0,0.0,0.0


А теперь воспользуемся библиотекой ```sklearn```

In [12]:
count_vect = CountVectorizer()
temp_matrix = count_vect.fit_transform(texts) # temp_matrix эта промежуточная матрица, понадобится в следующем кейсе,
                                              # для вычисления tf-idf
matrix_counts = temp_matrix.toarray()         # в данной матрице хранятся 

In [17]:
pd.DataFrame(temp_matrix.toarray())

,0,1,2,3,4,5,6
0,0,3,1,0,0,1,0
1,1,0,0,0,1,2,1
2,0,0,0,1,1,0,0


In [18]:
count_vect.vocabulary_

{'все': 0, 'мама': 1, 'мыла': 2, 'очень': 3, 'просто': 4, 'рама': 5, 'это': 6}

Визуализируем красиво ```matrix_counts```

In [19]:
words = [x[0] for x in sorted(count_vect.vocabulary_.items(), key=lambda x: x[1])] # список слов, 
                                                                                   # чтобы сделать красивую шапку
pd.DataFrame(matrix_counts, columns=words)  # при создании DataFrame передадим подготовленный список слов

,все,мама,мыла,очень,просто,рама,это
0,0,3,1,0,0,1,0
1,1,0,0,0,1,2,1
2,0,0,0,1,1,0,0


In [20]:
#tf-idf слово ПРОСТО
import math
1*math.log(3/2)

0.4054651081081644

# Что такое TF-IDF?

## TF - term frequency

1) Просто частотность, что мы считали выше (https://en.wikipedia.org/wiki/Tf–idf)

In [39]:
display(Math(r'f_{t,d}'))

<IPython.core.display.Math object>

2) TF по версии российской википедии (https://ru.wikipedia.org/wiki/TF-IDF) (!!!)

In [40]:
display(Math(r'\mathrm{tf}(t,d) = \frac{f_{t,d}}{\sum_k f_{t_k,d}}'))

<IPython.core.display.Math object>

3) Бинарная встречаемость. *0 - не было слово в тексте, 1 - было слово в тесте*

In [167]:
display(Math(r'\mathrm{tf}(t,d) = \left\{ \begin{matrix} 0, f_{t,d} = 0 \\ 1, f_{t,d} ≥ 1 \end{matrix} \right.'))

<IPython.core.display.Math object>

4) Логорифм от частоты +1

In [168]:
display(Math(r'\mathrm{tf}(t,d) = \log(1 + f_{t,d})'))

<IPython.core.display.Math object>

5) Нормированная частота (double normalization)

In [169]:
display(Math(r'K + (1 - K) \frac { f_{t,d} }{\max({f_{t_i,d}})}'))

<IPython.core.display.Math object>

Подробности https://en.wikipedia.org/wiki/Tf–idf

## IDF - inverse document frequency

1) 'Стандарт', логарифм отношения числа всех документов к числу документов, содержащих данное слово

In [41]:
display(Math(r'\mathrm{idf}(t,D)=\log\frac{N}{|\{d\in D:t\in d\}|}\
             =\log\frac{|D|}{|(d_{i}\supset t_{i})|}=\log\frac{N}{n_t}'))

<IPython.core.display.Math object>

2) Сглаженный IDF

In [172]:
display(Math(r'\mathrm{idf}(t,D)=\log(1+\frac{N}{n_t})'))

<IPython.core.display.Math object>

3) IDF с использованием максимального значения DF

In [174]:
display(Math(r'\log\left(1 + \frac {\max_t n_t} {n_t}\right)'))

<IPython.core.display.Math object>

4) Вероятностная обратная встречаемость

In [175]:
display(Math(r'\log\frac {N - n_t} {n_t}'))

<IPython.core.display.Math object>

# Различные библиотеки для вычисления TF-IDF

## sklearn

Документация http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer

Исходный код https://github.com/scikit-learn/scikit-learn/blob/bb592f3865f02f1d6bf9dedce1a2554fa0ada800/sklearn/feature_extraction/text.py#L901

In [ ]:
idf = (log(N/(n)+ 1) + 1)

In [27]:
tfidf_transformer = TfidfTransformer()  # обратить внимание на ```norm``` и ```smooth_idf```
matrix_tfidf = tfidf_transformer.fit_transform(temp_matrix).toarray()
pd.DataFrame(matrix_tfidf, columns=words)

,все,мама,мыла,очень,просто,рама,это
0,0.000000,0.922383,0.307461,0.000000,0.000000,0.233832,0.000000
1,0.452123,0.000000,0.000000,0.000000,0.343851,0.687703,0.452123
2,0.000000,0.000000,0.000000,0.795961,0.605349,0.000000,0.000000


In [44]:
pd.DataFrame([tfidf_transformer.idf_]*3, columns=words)  # построим только ```idf```

,все,мама,мыла,очень,просто,рама,это
0,2.098612,2.098612,2.098612,2.098612,1.405465,1.405465,2.098612
1,2.098612,2.098612,2.098612,2.098612,1.405465,1.405465,2.098612
2,2.098612,2.098612,2.098612,2.098612,1.405465,1.405465,2.098612


## NLTK

Исходный код https://github.com/nltk/nltk/blob/7ba46b9d52ed0c03bf806193f38d8c0e9bd8a9b4/nltk/text.py#L531

In [30]:
import nltk

In [34]:
a = nltk.text.TextCollection(texts)

In [45]:
a.tf('и', 'лавировали лавировали да не вылавировались')

0.14285714285714285

In [44]:
3/42

0.07142857142857142

In [64]:
3/len('корабли лавировали лавировали да не вылавировали')

0.0625

In [65]:
'корабли лавировали лавировали да не вылавировали'.count('ли')

4

In [28]:
mytexts = nltk.TextCollection(texts)

NameError: name 'nltk' is not defined

In [66]:
mytexts.tf(u'ли', 'корабли лавировали лавировали да не вылавировали')

0.08333333333333333

## ---

## Будьте внимательны